## Quora Incinsere Questions Classification sử dụng Logistic Regression


In [ ]:
# import các thư viện cần thiết
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import warnings
warnings.filterwarnings("ignore")
# import spacy
import re
from tqdm import tqdm
import nltk
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

## 1. Phân tích dữ liệu
#### Trước hết ta load dữ liệu vào dataframe và in ra để quan sát. Mục đích của bước này là để quan sát được dữ liệu câu hỏi gồm những trường gì, các trường thuộc kiểu dữ liệu gì để có thể xử lý và quyết định hướng tiếp cận giải quyết vấn đề.

In [ ]:
train_raw = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
validation_data = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
train_raw

#### Dữ liệu
- 1306122 hàng × 3 cột
- Chưa nhìn thấy được phân bố phân lớp của câu hỏi
#### Các feature gồm id câu hỏi, câu hỏi, phân lớp của câu hỏi:
- qid: id độc nhất của câu hỏi, qid chắc sẽ không tham gia vào bước phân lớp câu hỏi nên có thể bỏ được
- question_text: dữ liệu câu hỏi, do trường này là duy nhất tác động trực tiếp vào phân lớp của câu hỏi nên cần phải thực hiện tiền xử lý
- target: phân lớp của câu hỏi, target = 0 với câu hỏi sincere và target = 1 với câu hỏi incinsere



In [ ]:
y = train_raw['target']
y.value_counts().plot(kind='bar', rot=0)

- Số lượng câu hỏi sincere nhiều hơn rất nhiều so với insincere

### Tạo thêm một số feature để dễ quan sát, có thể sử dụng sau từ dữ liệu câu hỏi:
#### Dữ liệu của bài toán chỉ có trường văn bản của câu hỏi nên cần phải tạo thêm một số các feature để có thể quan sát kỹ hơn. Trong đó các feature mới được tạo ra là:
- số từ
- số từ độc nhất
- số ký tự đặc biệt
- số từ in hoa
- số từ không in hoa
- in hoa đầu từ

In [ ]:
def create_features(df_):
    
    df_["nb_words"] = df_["question_text"].apply(lambda x: len(x.split())) # số từ
    df_["nb_unique_words"] = df_["question_text"].apply(lambda x: len(set(str(x).split()))) # độ dài set
    df_["nb_chars"] = df_["question_text"].apply(lambda x: len(str(x))) # số ký tự
    df_['spe_chars'] = df_['question_text'].str.findall(r'[^a-zA-Z0-9 ]').str.len()
    df_["nb_uppercase"] = df_["question_text"].apply(lambda x : len([nu for nu in str(x).split() if nu.isupper()]))
    df_["nb_lowercase"] = df_["question_text"].apply(lambda x : len([nl for nl in str(x).split() if nl.islower()]))
    df_["nb_title"] = df_["question_text"].apply(lambda x : len([nl for nl in str(x).split() if nl.istitle()]))

    return df_

train_features = create_features(train_raw)

## Dữ liệu câu hỏi sincere

In [ ]:
train_features[train_features['target'] == 0].describe().round(1)

## Dữ liệu câu hỏi insincere

In [ ]:
train_features[train_features['target'] == 1].describe().round(1)

### Nhận xét:
- Câu hỏi toxic có trung bình(mean) của số từ, số lượng ký tự nhiều hơn câu hỏi non toxic 
- Có thể sử dụng các feature này vào model (đã thử và không có hiệu quả với các mô hình tuyến tính)
- Có thể sử dụng vector đếm từ, về căn bản vector đếm từ có khả năng lưu lại các feature như số lượng từ, số lượng từ độc nhất
- Các câu hỏi có số lượng ký tự đặc biệt hoặc số lượng từ đạt max thường có giá trị lớn hơn nhiều so với trung bình nên cần phải xem xét thêm

In [ ]:
## Câu hỏi insincere
print(train_raw['question_text'][(train_raw['target']==1)].sample(10).values)

### Nhận xét:
- Các câu hỏi insincere thường có các từ, cụm từ mang nghĩa xấu, ít khi phụ thuộc vào ngữ pháp
- Có thể sử dụng vector đếm từ vì không cần giữ lại ngữ pháp câu
- Các ký tự đặc biệt, chữ số, đường dẫn, in hoa hay in thường không ảnh hưởng nhiều đến phân lớp của câu hỏi nên có thể bỏ

#### Xem xét thêm về dữ liệu câu hỏi insincere
- Số lượng ký tự, ký tự đặc biệt trong câu hỏi
- Tần suất các từ trong câu hỏi

In [ ]:
plt.hist(train_features['nb_chars'][train_features['target'] == 1], bins=[0,50,100,150,200,250,300,400,500,1000])
plt.title('number of characters')
plt.show()

In [ ]:
plt.hist(train_features['spe_chars'], bins=[1,2,5,10,20,50])
plt.title('special characters')
plt.show()

- Hầu hết các câu hỏi insincere có số lượng ký tự nằm trong khoảng [0,300] và số lượng ký tự đặc biệt từ [0,10]. 
- Như vậy có rất nhiều dữ liệu nhiễu do từ bảng thống kê trên, câu hỏi có số lượng ký tự nhiều nhất là 1017 chữ và 411 ký tự.

### Sử dụng wordcloud để xem tần suất của các từ trong câu hỏi insincere

In [ ]:
wordcloud = WordCloud(width=800, height=600, collocations=False).generate(" ".join(train_raw['question_text'][train_raw['target']==1]))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.imshow(wordcloud,interpolation='bilinear')
plt.show()

- Các câu hỏi insincere thường có nhiều các từ mang nghĩa xấu
- Tuy nhiên một số các từ không mang nghĩa xấu có tần suất cao như people, will, many, much. Các từ này thuộc stopwords, tức là các từ cần thiết trong ngữ pháp nhưng không mang lại nhiều ý nghĩa khi xét từng từ riêng lẻ.

### Xem xét dữ liệu nhiễu, biên
- Từ các bảng trên, ta nhận thấy một số câu hỏi có số lượng từ, ký tự, ký tự đặc biệt nhiều hơn rất nhiều so với mean của toàn tập nên cần xem xét để bỏ

In [ ]:
print(train_features['question_text'][(train_raw['target']==1) & (train_features['nb_chars']>600.0)].values)
print(train_features['question_text'][(train_raw['target']==1) & (train_features['spe_chars']>30.0)].values)

### Nhận xét:
- Hầu hết các câu hỏi có nhiều ký tự đặc biệt là câu hỏi chứa công thức toán hoặc chữ tượng hình
- Một số câu hỏi liên quan đến toán bị xếp vào insincere, có thể là nhiễu nên bỏ được

## 2. Xử lý dữ liệu câu hỏi
### Sau bước phân tích dữ liệu, ta nhận thấy dữ liệu cần phải được xử lý vì các vấn đề:
- Tập dataset có tỉ lệ câu hỏi sincere:insincere là 15:1, không cân bằng
- Phải xử lý dữ liệu thô của văn bản do có chứa các ký tự đặc biệt, các từ stopwords không hữu ích,...
- Loại bỏ các câu hỏi có số lượng ký tự đặc biệt, số lượng từ lớn hơn nhiều so với trung bình.

### Bỏ các câu hỏi có số kí tự, số kí tự đặc biệt vượt ngưỡng
- Dữ liệu câu hỏi sẽ được lọc để cắt bớt phần biên

In [ ]:
# train_features_filtered = train_features.drop(train_features[(train_features['nb_chars'] >= 600) & (train_features['nb_unique_words']>35.0)].index)
train_features_filtered = train_features[(train_features['nb_chars']<600.0) & (train_features['nb_words']<70.0) & (train_features['spe_chars']<12.0)]

train_features_filtered.describe().round(1)

### Nhận xét:
- Đã bỏ 3494 hàng
- Số ký tự nhiều nhất là 300, ký tự đặc biệt là 11 sau khi bỏ 3494 câu hỏi, như vậy vẫn giữ được phần lớn số lượng câu hỏi nhưng bỏ được các câu hỏi nhiễu.

## Resample
- Số lượng câu hỏi sincere nhiều hơn rất nhiều so với câu hỏi insincere (gấp hơn 15 lần), dataset không cân bằng, gây ra overfit trên câu hỏi sincere, làm cho accuracy khi dự đoán câu hỏi insincere thấp
- Cần resample lại dataset cho cân bằng 
- Sau khi chia theo nhiều tỉ lệ, tỉ lệ 4:1 cho kết quả tốt nhất

In [ ]:
# Resampling
from sklearn.utils import resample

# sincere = train_raw[train_raw.target == 0]
# insincere = train_raw[train_raw.target == 1]

sincere = train_features_filtered[train_features_filtered.target == 0]
insincere = train_features_filtered[train_features_filtered.target == 1]

# Tỉ lệ 1:1
# x = pd.concat([resample(sincere,
#                      replace = False,
#                      n_samples = len(insincere)), insincere])

# Tỉ lệ 2:1
# x = pd.concat([resample(sincere,
#                      replace = True,
#                      n_samples = len(insincere)*2), insincere])

# Tỉ lệ 3:1
# x = pd.concat([resample(sincere,
#                      replace = True,
#        n_samples = len(insincere)*3), insincere])

# 4:1
x = pd.concat([resample(sincere,
                     replace = True,
                     n_samples = len(insincere)*4), insincere])


In [ ]:
y = x['target']
y.value_counts().plot(kind='bar', rot=0)

### Nhận xét: Tập train giờ đã cân bằng hơn

## Xử lý dữ liệu văn bản của câu hỏi
#### Từ kết quả của bước phân tích dữ liệu, ta có thể loại bỏ khỏi câu hỏi các dữ liệu không cần thiết và chuyển một số dữ liệu về dạng nguyên gốc:
- Bỏ đường link
- Loại bỏ ký tự đặc biệt
- Chuyển các từ cùng biến thể của một từ về một từ duy nhất
- Chuyển dạng rút gọn từ thành nguyên bản
- Bỏ chữ số
- Bỏ công thức toán trong tag latex
- Bỏ stopword

In [ ]:
# Bỏ đường link
def clean_tag(x):
    if 'http' in x or 'www' in x:
        x = re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '[url]', x) #replacing with [url]
    return x

In [ ]:
# Chuyển dạng rút gọn từ thành nguyên bản

contraction_mapping = {"We'd": "We had", "That'd": "That had", "AREN'T": "Are not", "HADN'T": "Had not", "Could've": "Could have", "LeT's": "Let us", "How'll": "How will", "They'll": "They will", "DOESN'T": "Does not", "HE'S": "He has", "O'Clock": "Of the clock", "Who'll": "Who will", "What'S": "What is", "Ain't": "Am not", "WEREN'T": "Were not", "Y'all": "You all", "Y'ALL": "You all", "Here's": "Here is", "It'd": "It had", "Should've": "Should have", "I'M": "I am", "ISN'T": "Is not", "Would've": "Would have", "He'll": "He will", "DON'T": "Do not", "She'd": "She had", "WOULDN'T": "Would not", "She'll": "She will", "IT's": "It is", "There'd": "There had", "It'll": "It will", "You'll": "You will", "He'd": "He had", "What'll": "What will", "Ma'am": "Madam", "CAN'T": "Can not", "THAT'S": "That is", "You've": "You have", "She's": "She is", "Weren't": "Were not", "They've": "They have", "Couldn't": "Could not", "When's": "When is", "Haven't": "Have not", "We'll": "We will", "That's": "That is", "We're": "We are", "They're": "They' are", "You'd": "You would", "How'd": "How did", "What're": "What are", "Hasn't": "Has not", "Wasn't": "Was not", "Won't": "Will not", "There's": "There is", "Didn't": "Did not", "Doesn't": "Does not", "You're": "You are", "He's": "He is", "SO's": "So is", "We've": "We have", "Who's": "Who is", "Wouldn't": "Would not", "Why's": "Why is", "WHO's": "Who is", "Let's": "Let us", "How's": "How is", "Can't": "Can not", "Where's": "Where is", "They'd": "They had", "Don't": "Do not", "Shouldn't":"Should not", "Aren't":"Are not", "ain't": "is not", "What's": "What is", "It's": "It is", "Isn't":"Is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def clean_contractions(x):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        x = x.replace(s, "'")
    
    x = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in x.split(" ")])
    return x

In [ ]:
# bỏ stopword
def remove_stopwords(x):
  x = [word for word in x.split() if word not in STOPWORDS]
  x = ' '.join(x)
  return x

In [ ]:
# Chuyển các từ cùng biến thể của một từ về một từ duy nhất
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
l = WordNetLemmatizer()

def lemmatize_text(x):
    x = ' '.join([l.lemmatize(word) for word in word_tokenize(x)])
    return x

In [ ]:
# Loại bỏ ký tự đặc biệt
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', 
        '•', '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 
        '█', '…', '“', '★', '”', '–', '●', '►', '−', '¢', '¬', '░', '¡', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
        '—', '‹', '─', '▒', '：', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '¸', '⋅', '‘', '∞', 
        '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '・', '╦', '╣', '╔', '╗', '▬', '❤', '≤', '‡', '√', '◄', '━', 
        '⇒', '▶', '≥', '╝', '♡', '◊', '。', '✈', '≡', '☺', '✔', '↵', '≈', '✓', '♣', '☎', '℃', '◦', '└', '‟', '～', '！', '○', 
        '◆', '№', '♠', '▌', '✿', '▸', '⁄', '□', '❖', '✦', '．', '÷', '｜', '┃', '／', '￥', '╠', '↩', '✭', '▐', '☼', '☻', '┐', 
        '├', '«', '∼', '┌', '℉', '☮', '฿', '≦', '♬', '✧', '〉', '－', '⌂', '✖', '･', '◕', '※', '‖', '◀', '‰', '\x97', '↺', 
        '∆', '┘', '┬', '╬', '،', '⌘', '⊂', '＞', '〈', '⎙', '？', '☠', '⇐', '▫', '∗', '∈', '≠', '♀', '♔', '˚', '℗', '┗', '＊', 
        '┼', '❀', '＆', '∩', '♂', '‿', '∑', '‣', '➜', '┛', '⇓', '☯', '⊖', '☀', '┳', '；', '∇', '⇑', '✰', '◇', '♯', '☞', '´', 
        '↔', '┏', '｡', '◘', '∂', '✌', '♭', '┣', '┴', '┓', '✨', '\xa0', '˜', '❥', '┫', '℠', '✒', '［', '∫', '\x93', '≧', '］', 
        '\x94', '∀', '♛', '\x96', '∨', '◎', '↻', '⇩', '＜', '≫', '✩', '✪', '♕', '؟', '₤', '☛', '╮', '␊', '＋', '┈', '％', 
        '╋', '▽', '⇨', '┻', '⊗', '￡', '।', '▂', '✯', '▇', '＿', '➤', '✞', '＝', '▷', '△', '◙', '▅', '✝', '∧', '␉', '☭', 
        '┊', '╯', '☾', '➔', '∴', '\x92', '▃', '↳', '＾', '׳', '➢', '╭', '➡', '＠', '⊙', '☢', '˝', '∏', '„', '∥', '❝', '☐', 
        '▆', '╱', '⋙', '๏', '☁', '⇔', '▔', '\x91', '➚', '◡', '╰', '\x85', '♢', '˙', '۞', '✘', '✮', '☑', '⋆', 'ⓘ', '❒', 
        '☣', '✉', '⌊', '➠', '∣', '❑', '◢', 'ⓒ', '\x80', '〒', '∕', '▮', '⦿', '✫', '✚', '⋯', '♩', '☂', '❞', '‗', '܂', '☜', 
        '‾', '✜', '╲', '∘', '⟩', '＼', '⟨', '·', '✗', '♚', '∅', 'ⓔ', '◣', '͡', '‛', '❦', '◠', '✄', '❄', '∃', '␣', '≪', '｢', 
        '≅', '◯', '☽', '∎', '｣', '❧', '̅', 'ⓐ', '↘', '⚓', '▣', '˘', '∪', '⇢', '✍', '⊥', '＃', '⎯', '↠', '۩', '☰', '◥', 
        '⊆', '✽', '⚡', '↪', '❁', '☹', '◼', '☃', '◤', '❏', 'ⓢ', '⊱', '➝', '̣', '✡', '∠', '｀', '▴', '┤', '∝', '♏', 'ⓐ', 
        '✎', ';', '␤', '＇', '❣', '✂', '✤', 'ⓞ', '☪', '✴', '⌒', '˛', '♒', '＄', '✶', '▻', 'ⓔ', '◌', '◈', '❚', '❂', '￦', 
        '◉', '╜', '̃', '✱', '╖', '❉', 'ⓡ', '↗', 'ⓣ', '♻', '➽', '׀', '✲', '✬', '☉', '▉', '≒', '☥', '⌐', '♨', '✕', 'ⓝ', 
        '⊰', '❘', '＂', '⇧', '̵', '➪', '▁', '▏', '⊃', 'ⓛ', '‚', '♰', '́', '✏', '⏑', '̶', 'ⓢ', '⩾', '￠', '❍', '≃', '⋰', '♋', 
        '､', '̂', '❋', '✳', 'ⓤ', '╤', '▕', '⌣', '✸', '℮', '⁺', '▨', '╨', 'ⓥ', '♈', '❃', '☝', '✻', '⊇', '≻', '♘', '♞', 
        '◂', '✟', '⌠', '✠', '☚', '✥', '❊', 'ⓒ', '⌈', '❅', 'ⓡ', '♧', 'ⓞ', '▭', '❱', 'ⓣ', '∟', '☕', '♺', '∵', '⍝', 'ⓑ', 
        '✵', '✣', '٭', '♆', 'ⓘ', '∶', '⚜', '◞', '்', '✹', '➥', '↕', '̳', '∷', '✋', '➧', '∋', '̿', 'ͧ', '┅', '⥤', '⬆', '⋱', 
        '☄', '↖', '⋮', '۔', '♌', 'ⓛ', '╕', '♓', '❯', '♍', '▋', '✺', '⭐', '✾', '♊', '➣', '▿', 'ⓑ', '♉', '⏠', '◾', '▹', 
        '⩽', '↦', '╥', '⍵', '⌋', '։', '➨', '∮', '⇥', 'ⓗ', 'ⓓ', '⁻', '⎝', '⌥', '⌉', '◔', '◑', '✼', '♎', '♐', '╪', '⊚', 
        '☒', '⇤', 'ⓜ', '⎠', '◐', '⚠', '╞', '◗', '⎕', 'ⓨ', '☟', 'ⓟ', '♟', '❈', '↬', 'ⓓ', '◻', '♮', '❙', '♤', '∉', '؛', 
        '⁂', 'ⓝ', '־', '♑', '╫', '╓', '╳', '⬅', '☔', '☸', '┄', '╧', '׃', '⎢', '❆', '⋄', '⚫', '̏', '☏', '➞', '͂', '␙', 
        'ⓤ', '◟', '̊', '⚐', '✙', '↙', '̾', '℘', '✷', '⍺', '❌', '⊢', '▵', '✅', 'ⓖ', '☨', '▰', '╡', 'ⓜ', '☤', '∽', '╘', 
        '˹', '↨', '♙', '⬇', '♱', '⌡', '⠀', '╛', '❕', '┉', 'ⓟ', '̀', '♖', 'ⓚ', '┆', '⎜', '◜', '⚾', '⤴', '✇', '╟', '⎛', 
        '☩', '➲', '➟', 'ⓥ', 'ⓗ', '⏝', '◃', '╢', '↯', '✆', '˃', '⍴', '❇', '⚽', '╒', '̸', '♜', '☓', '➳', '⇄', '☬', '⚑', 
        '✐', '⌃', '◅', '▢', '❐', '∊', '☈', '॥', '⎮', '▩', 'ு', '⊹', '‵', '␔', '☊', '➸', '̌', '☿', '⇉', '⊳', '╙', 'ⓦ', 
        '⇣', '｛', '̄', '↝', '⎟', '▍', '❗', '״', '΄', '▞', '◁', '⛄', '⇝', '⎪', '♁', '⇠', '☇', '✊', 'ி', '｝', '⭕', '➘', 
        '⁀', '☙', '❛', '❓', '⟲', '⇀', '≲', 'ⓕ', '⎥', '\u06dd', 'ͤ', '₋', '̱', '̎', '♝', '≳', '▙', '➭', '܀', 'ⓖ', '⇛', '▊', 
        '⇗', '̷', '⇱', '℅', 'ⓧ', '⚛', '̐', '̕', '⇌', '␀', '≌', 'ⓦ', '⊤', '̓', '☦', 'ⓕ', '▜', '➙', 'ⓨ', '⌨', '◮', '☷', 
        '◍', 'ⓚ', '≔', '⏩', '⍳', '℞', '┋', '˻', '▚', '≺', 'ْ', '▟', '➻', '̪', '⏪', '̉', '⎞', '┇', '⍟', '⇪', '▎', '⇦', '␝', 
        '⤷', '≖', '⟶', '♗', '̴', '♄', 'ͨ', '̈', '❜', '̡', '▛', '✁', '➩', 'ா', '˂', '↥', '⏎', '⎷', '̲', '➖', '↲', '⩵', '̗', '❢', 
        '≎', '⚔', '⇇', '̑', '⊿', '̖', '☍', '➹', '⥊', '⁁', '✢']

def clean_punct(x):
  for punct in puncts:
    if punct in x:
      x = x.replace(punct, f' {punct} ')
  return x

In [ ]:
# bỏ chữ số
def clean_numbers(x):
    x = re.sub(r'(\d+)([a-zA-Z])', '\g<1> \g<2>', x)
    x = re.sub(r'(\d+) (th|st|nd|rd) ', '\g<1>\g<2> ', x)
    x = re.sub(r'(\d+),(\d+)', '\g<1>\g<2>', x)
    return x

In [ ]:
# thay các tag latex
def clean_latex_tag(x):
    corr_t = []
    for t in x.split(" "):
        t = t.strip()
        if t != '':
            corr_t.append(t)
    x = ' '.join(corr_t)
    x = re.sub('(\[ math \]).+(\[ / math \])', 'math formula', x)
    return x

In [ ]:
#gộp các hàm xử lý lại
def data_cleaning(x):
    x = clean_tag(x)
    x = clean_contractions(x)
    x = clean_punct(x)
    x = lemmatize_text(x)
    x = clean_latex_tag(x)
    x = clean_numbers(x)
    x = remove_stopwords(x)
    return x

In [ ]:
#xử lý dữ liệu câu hỏi trên tập train và validation
x['question_text'] = x['question_text'].progress_map(lambda x: data_cleaning(x))
validation_data['question_text']=validation_data['question_text'].progress_map(lambda x: data_cleaning(x))

## 3. Đưa dữ liệu vào mô hình phân lớp
- Từ bước phân tích dữ liệu, ta nhận thấy các câu hỏi insincere thường chứa các từ ngữ mang nghĩa xấu, không phụ thuộc vào ngữ pháp nên chọn hướng tiếp cận sử dụng CountVectorizer là vector đếm số lượng từ.
- Bước xử lý dữ liệu cũng đã thu nhỏ được tập từ vựng khi sử dụng CountVectorizer qua việc loại bỏ dữ liệu biên và xử lý dữ liệu văn bản.

## Chia tập test, train

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x['question_text'], x['target'], test_size=0.2, random_state=0)
print('x_train: ', x_train.shape, y_train.shape)
print('x_test: ',x_test.shape, y_test.shape)

## Tạo ra các vector đếm từ tập train, test
- Vector đếm từ: chuyển câu thành vector chứa các từ và số lần xuất hiện của từ đó trong câu
- Phân lớp câu hỏi không phụ thuộc vào ngữ pháp
- Học trên tập từ vựng của toàn bộ tập train và test do ở bước test, vector đếm có thể phải mã hoá những từ có ở tập test mà không nằm trong tập train

In [ ]:
vectorizer = CountVectorizer()
# Học trên tập từ vựng của toàn bộ tập train và validation
vectorizer.fit(list(x['question_text'].values)+ list(validation_data['question_text'].values))
# Tạo vector đếm cho tập train, test, validation dựa trên tập từ vựng đã học
x_tr = vectorizer.transform(x_train) 
x_te = vectorizer.transform(x_test)
x_val = vectorizer.transform(validation_data['question_text'])
print(x_tr.shape)
print(x_te.shape)
print(x_val.shape)

### Sử dụng Logistic Regression để phân lớp câu hỏi từ các vector đếm
- Sau khi có được vector đếm, chỉ cần đưa vector vào model để train
- Sử dụng Logistic Regression vì model đơn giản, hiệu quả hơn so với các model khác: Random Forest Classifier, Naive Bayes, SVM

In [ ]:
# import
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression

### Tối ưu tham số bằng GridSearchCV
- Đầu vào của model có tham số C, sử dụng GridSearchCV nhằm tìm ra tham số C tốt nhất cho model

In [ ]:
# Tìm kiếm tham số 
from sklearn.model_selection import  GridSearchCV
params = {'C': [0.1, 1, 2, 3, 4, 5, 10]}

gridsearch = GridSearchCV(LogisticRegression(), params, scoring='f1', n_jobs=-1, verbose=1)
gridsearch.fit(x_tr, y_train)
print(gridsearch.best_params_)

### Chạy mô hình trên tham số đã tìm được

In [ ]:
%%time
model = LogisticRegression(C=gridsearch.best_params_['C'])
## Chạy mô hình
print(f"Running Logistic Regression")
model.fit(x_tr, y_train)

train_predictions = model.predict(x_tr)
train_acc = accuracy_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions)
print(f"Train accuracy: {train_acc:.2%}, F1: {train_f1:.4f}") 
test_predictions = model.predict(x_te)
test_acc = accuracy_score(y_test, test_predictions) 
test_f1 = f1_score(y_test, test_predictions) 
print(f"Test accuracy:  {test_acc:.2%}, F1: {test_f1:.4f}")

In [ ]:
# In ra confusion matrix, classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix
sns.set(font_scale=1.4)
sns.heatmap(pd.DataFrame(confusion_matrix(y_test, test_predictions), range(2),range(2)), annot=True, fmt='g')
print(classification_report(y_test, test_predictions))

### Nhận xét:
- F1 và accuracy đã khá gần nhau sau khi cân bằng dataset
- F1 của model khi dự đoán câu hỏi insincere tăng lên đáng kể sau khi cân bằng dataset
- Accuracy của model khi dự đoán câu hỏi sincere giảm nhưng không đáng kể sau khi cân bằng dataset
- Không bị overfit trên câu hỏi sincere

## 4. Submission
- Sau khi train xong, chạy mô hình trên tập validation và lưu lại để submit

In [ ]:
# Submission

validation_predictions = model.predict(x_val)
submission = pd.DataFrame({'qid':validation_data['qid'], 'prediction':validation_predictions })
submission.to_csv('submission.csv', index=False)
submission

### Nhận xét:
- Kết quả F1 trên tập validation: ~0.6
- Điểm F1 tập validation lệch nhiều so với F1 tập test và tập train nhưng tương đối cao với mô hình tuyến tính